<a href="https://colab.research.google.com/github/Fis428/PC03/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝相依套件
!pip install -q flask pyngrok flask_ngrok line-bot-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.0 MB/s eta 0:00:00


In [ ]:

import os

# TODO: 將引號內替換成你的實際值
os.environ['LINE_CHANNEL_ACCESS_TOKEN'] = '你的 LINE_CHANNEL_ACCESS_TOKEN'
os.environ['LINE_CHANNEL_SECRET']      = '你的 LINE_CHANNEL_SECRET'
os.environ['GOOGLE_API_KEY']           = '你的 GOOGLE_API_KEY'
os.environ['NGROK_AUTHTOKEN']          = '你的 NGROK_AUTHTOKEN'

# 1. 安裝必要套件（如已安裝過可略過）
!pip install -q flask pyngrok flask_ngrok line-bot-sdk google-generativeai

# 2. 載入套件
from flask import Flask, request, abort
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

import google.generativeai as genai

# 3. 從環境變數讀取金鑰
LINE_TOKEN  = os.environ['LINE_CHANNEL_ACCESS_TOKEN']
LINE_SECRET = os.environ['LINE_CHANNEL_SECRET']
GOOGLE_KEY  = os.environ['GOOGLE_API_KEY']
NGROK_TOKEN = os.environ['NGROK_AUTHTOKEN']

# 4. Flask 與 LINE、Gemini 初始化
app = Flask(__name__)

configuration = Configuration(access_token=LINE_TOKEN)
handler       = WebhookHandler(LINE_SECRET)

genai.configure(api_key=GOOGLE_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

def ask_gemini(question: str) -> str:
    return model.generate_content(question).text

# 5. ngrok 啟動（Colab 上用）
def start_ngrok():
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(5000)
    print("ngrok URL:", public_url)
    run_with_ngrok(app)

# 6. 設定 LINE Webhook 路由
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body      = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

# 7. 處理收到的訊息：文字、貼圖、圖片、影片、位置或交給 Gemini
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_input = event.message.text.strip().lower()

    # 根據特定指令回不同訊息類型
    if user_input == 'sticker':
        reply = StickerMessage(package_id='446', sticker_id='1989')
    elif user_input == 'image':
        reply = ImageMessage(
            original_content_url='https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg',
            preview_image_url='https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg'
        )
    elif user_input == 'video':
        reply = VideoMessage(
            original_content_url='https://cdn.pixabay.com/video/2021/07/08/80724-573496156_large.mp4',
            preview_image_url='https://cdn.pixabay.com/photo/2017/02/11/14/45/taipei-taiwan-2057818_1280.jpg'
        )
    elif user_input == 'location':
        reply = LocationMessage(
            title='Taipei 101',
            address='110台北市信義區信義路五段7號',
            latitude=25.034199559170606,
            longitude=121.56456035396677
        )
    else:
        # 其餘文字交給 Gemini AI 回答
        ai_reply = ask_gemini(event.message.text)
        reply = TextMessage(text=ai_reply)

    # 回覆 LINE
    with ApiClient(configuration) as api_client:
        line_api = MessagingApi(api_client)
        line_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# 8. 啟動應用
if __name__ == "__main__":
    # 若在 Colab，使用 ngrok；否則直接跑 Flask
    try:
        start_ngrok()
    except Exception:
        app.run(port=5000)


ERROR:pyngrok.process.ngrok:t=2025-06-16T02:26:25+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: 你的 NGROK_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-16T02:26:25+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: 你的 NGROK_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-16T02:26:25+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a proper n

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
